In [1]:
%cd /home/jupyter/lazaro_projects/coe-iaaa-kfp-pipelines-executions

/home/jupyter/lazaro_projects/coe-iaaa-kfp-pipelines-executions


In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import os
from dotenv import load_dotenv
# os.environ.clear()
load_dotenv('env', override=True)
load_dotenv('env.secret', override=True) # Ignored in git
from aikfp.src.executes import *
from aikfp.ingestamg import *
from aikfp.aimg import transformer_skip,register_mg_exp_v2
from aikfp.constants import mtd_pipeline
from aikfp.src.automl.pipeline_cfg import generate_metadata

In [12]:
df,_l_m=get_list_kfpipelines(False)

,id_pipeline,version,name_pipeline,source,name_subproject,state_pipeline,createTime,startTime,endTime,updateTime,n_nodes,paths_parquets,paths_artifacts,name_target,paths_models,paths_dataframes_train,period_of_training
0,gestion-leads-0-2-0-prediction-20220616120120,0-2-0,20220616120120,prediction,gestion-leads,PIPELINE_STATE_SUCCEEDED,2022-06-16 12:01:20.481162+00:00,2022-06-16 12:01:21.214591+00:00,2022-06-16 12:07:43.468159+00:00,2022-06-16 12:07:43.468159+00:00,5.0,[gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/P...,[],{gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/M...,{gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/M...,[bq://SELECT * FROM `rs-nprd-dlk-ia-dev-aif-d3...,NaT
1,gestion-leads-0-2-0-prediction-20220617120108,0-2-0,20220617120108,prediction,gestion-leads,PIPELINE_STATE_SUCCEEDED,2022-06-17 12:01:09.124515+00:00,2022-06-17 12:01:10.041855+00:00,2022-06-17 12:06:31.929983+00:00,2022-06-17 12:06:31.929983+00:00,5.0,[gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/P...,[],{gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/M...,{gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/M...,[bq://SELECT * FROM `rs-nprd-dlk-ia-dev-aif-d3...,NaT
2,gestion-leads-0-2-0-prediction-20220618120057,0-2-0,20220618120057,prediction,gestion-leads,PIPELINE_STATE_SUCCEEDED,2022-06-18 12:00:58.239084+00:00,2022-06-18 12:00:58.975647+00:00,2022-06-18 12:06:23.925055+00:00,2022-06-18 12:06:23.925055+00:00,5.0,[gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/P...,[],{gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/M...,{gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/M...,[bq://SELECT * FROM `rs-nprd-dlk-ia-dev-aif-d3...,NaT


In [40]:
df.to_csv(f'gs://ue4_ndlk_nonprod_stg_gcs_iadev_adsto/tmp/Ronald/projects/coe-iaaaa-iniciatives-model-governance/inputs/df_mtd_vertex.csv',index=False)
pd.to_pickle(_l_m,f'gs://ue4_ndlk_nonprod_stg_gcs_iadev_adsto/tmp/Ronald/projects/coe-iaaaa-iniciatives-model-governance/inputs/list_ids.pkl')
for _id in ['gestion-leads']:
        print(_id)
        df_train,df_pred,models=read_artifacts_and_datasets(df,_id,_l_m)
        pd.to_pickle({'df_train':df_train,
             'df_pred':df_pred,
             'models':models},f'gs://ue4_ndlk_nonprod_stg_gcs_iadev_adsto/tmp/Ronald/projects/coe-iaaaa-iniciatives-model-governance/ml_real/{_id}.pkl')

gestion-leads


Downloading: 100%|██████████| 683119/683119 [00:02<00:00, 270265.87rows/s]


In [4]:
mf=pd.read_pickle('gs://ue4_ndlk_nonprod_stg_gcs_iadev_adsto/tmp/Ronald/projects/coe-iaaaa-iniciatives-model-governance/ml_real/gestion-leads.pkl')
mf['df_train']['periodo']='2022-06-01'

In [5]:
%cd /home/jupyter/lazaro_projects/all-projects/coe-iaaa-iniciatives-priorizacion-leads

/home/jupyter/lazaro_projects/all-projects/coe-iaaa-iniciatives-priorizacion-leads


In [6]:
import pandas as pd
mf['preprocess']=pd.read_pickle('gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/MODELS/lead_priorizacion_ami/0.2.0/preprocess.pkl')

In [11]:
pd.set_option('display.max_columns',None)
mf['df_train'].head(3)

,fechalead,nrodoc,idlead,tiempo_desde_ult_gest,tiempo_desde_llegada,duracion_total_gestiones,duracion_ult_gestion,edad,tiempo_desde_ult_cef,prom_tiempo_cef,tiempo_prom_entre_gest,tiempo_contacto_efectivo,tiempo_prom_entre_gest_ini,tiempo_desde_primer_cef,cuenta_marc_manual,tiempo_prom_entre_gest_cef,porc_nocontacto,tiempo_total_visitas_15dias,numero_gestion,cuenta_marc_predictivo,tiempo_prom_entre_gest_cef_ini,prom_tiempo_visita_15dias,finalizado,total_nocontactos,plataforma,interacciones_ds_15dias,canal_fin,interacciones_google_15dias,prom_tiempo_visita_3meses,total_interacciones_15dias,tiempo_total_visitas_3meses,interacciones_mobile_15dias,prom_tiempo_visita_1,interacciones_fds_15dias,total_whatsapp,interacciones_web_15dias,total_contactos_efectivos,agrup_nse,interacciones_pagado_15dias,total_interacciones_3meses,interacciones_prod_salud_15dias,interacciones_facebook_15dias,interacciones_ds_3meses,interacciones_mobile_3meses,interacciones_fds_1,interacciones_web_3meses,tiempo_total_visitas_1,interacciones_fds_3meses,interacciones_pagado_3meses,interacciones_other_3meses,interacciones_other_15dias,ult_nivel1,flag_no_contactado_last_1,finalizado_bins,rango_edad,numero_gestion_bins,tiempo_desde_llegada_horas_bins,tiempo_desde_ult_gest_min_bins,total_contactos_efectivos_bins,total_nocontactos_bins,porc_nocontacto_bins,prom_mar_manual,prom_mar_manual_bins,periodo,lead_priorizacionauc_ami_cls
0,2021-07-17 20:30:57+00:00,70293719,1612609,168.0,8462.136667,0.109167,1.450000,29,0.000000,0.000000,543.484444,0.000000,579.721944,0.000000,1,0.000000,100.000000,0.0,15,8,0.000000,0.0,1,12,2,0.0,1,0.0,141.0,0.0,708.0,0.0,0.0,0.0,0,0.0,0,2,0.0,43.0,0.0,0.0,42.0,43.0,0.0,43.0,0.0,1.0,19.0,0.0,0.0,ERROR DE TELÉFONO,0,1,2,5,8,5,0,6,5,11.111111,3,2022-06-01,0
1,2021-08-13 19:50:22+00:00,44966213,1661927,168.0,7814.813056,0.289444,1.083333,32,522.071667,1.833333,405.152222,14.783333,428.976389,7814.673889,4,911.575278,50.000000,0.0,18,11,911.592500,0.0,0,8,0,0.0,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,8,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONTACTO EFECTIVO,0,0,3,5,8,5,4,5,2,26.666667,4,2022-06-01,0
2,2021-08-18 07:02:46+00:00,07842713,1670298,168.0,7707.606389,1.138889,0.666667,80,494.295278,2.650000,73.605000,21.233333,74.354167,7657.241667,27,895.368056,91.304348,0.0,98,57,901.663889,0.0,0,84,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,8,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,CONTACTO EFECTIVO,0,0,6,7,8,5,4,8,5,32.142857,4,2022-06-01,1


In [21]:
mf['preprocess'].fit_transform(df_train)

KeyError: 'fec_nacimiento'

In [6]:
cfg=C.mtd_pipeline['gestion-leads']
cfg=generate_metadata(mf['df_train'],cfg)

In [7]:
mg=register_mg_exp_v2(mf['df_train'],
                      mf['de_pred'][1],
                      mf['models']['MODELS/lead_priorizacion_ami/0.2.0/model_lead_priorizacionauc_ami_cls.pkl'],
                      mf['preprocess'],
                      cfg,
                 #     C.mtd_pipeline['gestion-leads'],
                      source='kf_predict')

NameError: name 'dict_preprocess' is not defined

In [84]:
mf['models']['MODELS/lead_priorizacion_ami/0.2.0/model_lead_priorizacionauc_ami_cls.pkl']._n_classes#.feature_name()#.feature_importance()

AttributeError: 'Booster' object has no attribute '_n_classes'

In [ ]:
!gsutil cp -r  "gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/PACKAGES/gestion_leads/gestion_leads-0.2.0.tar.gz" "gestion-leads.tar.gz"

In [6]:
name_package='gestion-leads'
enviorment_vars = {env: os.environ[env] for env in config._envs}
fixed_enviorment_vars = enviorment_vars.__str__()
fixed_package_uri = 'gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/PACKAGES/gestion_leads/gestion_leads-0.1.0.tar.gz'
import os
fixed_enviorment_vars = eval(fixed_enviorment_vars)
for env, val in fixed_enviorment_vars.items():
    os.environ[env] = val
os.system(f'''
    gcloud config set project $GOOGLE_CLOUD_PROJECT
    export GH_TOKEN=$(gcloud secrets versions access latest --secret="$SECRET_GH_TOKEN")
    gsutil cp "{fixed_package_uri}" {name_package}.tar.gz
    pip install --quiet --no-warn-script-location {name_package}.tar.gz
    ''')


,tiempo_desde_ult_gest,tiempo_desde_llegada,duracion_total_gestiones,duracion_ult_gestion,edad,tiempo_desde_ult_cef,prom_tiempo_cef,tiempo_prom_entre_gest,tiempo_contacto_efectivo,tiempo_prom_entre_gest_ini,...,interacciones_ds_3meses,interacciones_mobile_3meses,interacciones_fds_1,interacciones_web_3meses,tiempo_total_visitas_1,interacciones_fds_3meses,interacciones_pagado_3meses,flag_no_contactado_last_1,interacciones_other_3meses,interacciones_other_15dias
0,168.0,8462.136667,0.109167,1.450000,29,0.000000,0.000000,543.484444,0.000000,579.721944,...,42.0,43.0,0.0,43.0,0.0,1.0,19.0,0,0.0,0.0
1,168.0,7814.813056,0.289444,1.083333,32,522.071667,1.833333,405.152222,14.783333,428.976389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
2,168.0,7707.606389,1.138889,0.666667,80,494.295278,2.650000,73.605000,21.233333,74.354167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
3,168.0,7548.940556,1.096944,1.983333,18,18.454444,6.350000,358.594444,30.000000,375.986111,...,6.0,0.0,0.0,6.0,682.0,0.0,0.0,0,0.0,0.0
4,168.0,6645.727778,0.498889,20.000000,32,496.837222,13.283333,1537.222500,26.566667,2049.626389,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17330,168.0,127.235833,0.075278,1.900000,49,0.000000,0.000000,11.818611,0.000000,15.354167,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
17331,168.0,127.155556,0.136667,0.000000,23,122.518333,7.700000,17.545278,7.700000,20.126944,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
17332,168.0,127.124167,0.088056,2.116667,43,0.000000,0.000000,6.255000,0.000000,6.587222,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0
17333,168.0,126.875556,0.424444,0.000000,38,97.355000,20.966667,12.061667,20.966667,13.024444,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,0.0,0.0


In [159]:
for x in l['churn-empresas-0-1-0-prediction-20220315235131']['pipelineSpec']['deploymentSpec']['executors'].keys():
    try:print(l['churn-empresas-0-1-0-prediction-20220315235131']['pipelineSpec']['deploymentSpec']['executors'][x]['container']['resources']['memoryLimit'])
    except: pass

{'memoryLimit': 60.0}


In [161]:
models['perveh_propension_vehpagoxkmplata_cls']
a=pd.read_pickle('gs://ue4_ndlk_nonprod_stg_gcs_iadev_artfsto/MODELS/perveh_propension_vehpagoxkmplata_cls/0.3.9/deciles_recomendador.pkl')
a

array([0.00415451, 0.07882314, 0.16342327, 0.24611263, 0.33844377,
       0.42216893, 0.48789144, 0.54461336, 0.59874432, 0.64847946,
       0.79115249])

In [194]:

df_train

,fechalead,fecha_ini_t,nrodoc,idlead,tiempo_desde_ult_gest,tiempo_desde_llegada,duracion_total_gestiones,duracion_ult_gestion,edad,tiempo_desde_ult_cef,prom_tiempo_cef,tiempo_prom_entre_gest,tiempo_contacto_efectivo,tiempo_prom_entre_gest_ini,tiempo_desde_primer_cef,cuenta_marc_manual,tiempo_prom_entre_gest_cef,porc_nocontacto,tiempo_total_visitas_15dias,numero_gestion,cuenta_marc_predictivo,tiempo_prom_entre_gest_cef_ini,prom_tiempo_visita_15dias,finalizado,total_nocontactos,plataforma,interacciones_ds_15dias,canal_fin,interacciones_google_15dias,prom_tiempo_visita_3meses,total_interacciones_15dias,tiempo_total_visitas_3meses,interacciones_mobile_15dias,prom_tiempo_visita_1,interacciones_fds_15dias,total_whatsapp,interacciones_web_15dias,total_contactos_efectivos,agrup_nse,interacciones_pagado_15dias,total_interacciones_3meses,interacciones_prod_salud_15dias,interacciones_facebook_15dias,interacciones_ds_3meses,interacciones_mobile_3meses,interacciones_fds_1,interacciones_web_3meses,tiempo_total_visitas_1,interacciones_fds_3meses,interacciones_pagado_3meses,interacciones_other_3meses,interacciones_other_15dias,ult_nivel1,cuenta_ganado,considerar,cuenta_ganado_lead
0,2020-09-28 11:27:05+00:00,2020-10-02 10:06:06+00:00,07362282,1007397,58575,340741,376.0,138.0,62,NaN,0,47027,0,56151,NaN,0,0,1,390.0,7,3,0,390.0,3,6,Comprar,38.0,Google Ads,38.0,390.0,38.0,390.0,0.0,390.0,0.0,0,38.0,0,A-C,38.0,38.0,2.0,0.0,38.0,0.0,0.0,38.0,390.0,0.0,38.0,0.0,0.0,NO CONTACTO,0,0,0
1,2020-09-28 11:42:38+00:00,2020-10-02 10:38:53+00:00,48124723,1007435,60187,341775,292.0,190.0,26,NaN,0,40226,0,46902,NaN,0,0,1,576.0,8,4,0,576.0,2,7,Cotizador,34.0,Google Ads,34.0,576.0,34.0,576.0,0.0,576.0,0.0,0,34.0,0,A-C,34.0,34.0,6.0,0.0,34.0,0.0,0.0,34.0,576.0,0.0,34.0,0.0,0.0,NO CONTACTO,0,0,0
2,2020-10-18 12:28:53+00:00,2020-11-02 15:09:26+00:00,08123750,1043349,430086,1305633,1457.0,516.0,53,430086.0,728,145924,1457,159739,1228783.0,0,798697,0,254.0,7,5,875547,130.0,1,4,Comprar,24.0,Facebook Ads,0.0,84.0,48.0,254.0,48.0,84.0,24.0,1,48.0,2,A-C,48.0,48.0,0.0,48.0,24.0,48.0,24.0,48.0,254.0,24.0,48.0,0.0,0.0,CONTACTO EFECTIVO,0,0,0
3,2020-10-21 06:21:18+00:00,2020-11-05 16:13:31+00:00,29629786,1048554,761674,1331533,1383.0,249.0,51,NaN,0,56985,0,62481,NaN,0,0,1,2007.0,11,7,0,1016.0,1,10,Lead Ads,178.0,Facebook Ads,178.0,669.0,178.0,2007.0,178.0,669.0,0.0,0,178.0,0,A-C,178.0,178.0,10.0,0.0,178.0,178.0,0.0,178.0,2007.0,0.0,178.0,0.0,0.0,NO CONTACTO,0,0,0
4,2020-10-28 22:29:11+00:00,2020-11-03 10:28:04+00:00,76595963,1063718,75841,475133,134.0,0.0,19,NaN,0,79858,0,91115,NaN,0,0,1,2816.0,6,2,0,1408.0,1,5,Cotizador,50.0,Google Ads,36.0,1408.0,50.0,2816.0,50.0,1408.0,0.0,0,50.0,0,None,36.0,50.0,8.0,0.0,50.0,50.0,0.0,50.0,2816.0,0.0,36.0,0.0,0.0,NO CONTACTO,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3269472,2020-07-31 04:47:06+00:00,2020-07-31 08:30:22+00:00,41200304,892378,0,13396,NaN,NaN,38,NaN,0,0,0,0,NaN,0,0,0,164.0,1,0,0,164.0,1,0,Lead Ads,0.0,Facebook Ads,0.0,0.0,32.0,0.0,32.0,0.0,32.0,0,32.0,0,A-C,32.0,1.0,0.0,32.0,1.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,None,0,0,0
3269473,2021-05-27 20:25:54+00:00,2021-05-31 14:09:28+00:00,10146461,1512720,4224,323014,685.0,0.0,66,NaN,0,28980,0,31848,NaN,0,0,1,565.0,12,6,0,565.0,2,11,Comprar,32.0,Google Ads,32.0,0.0,32.0,0.0,32.0,NaN,0.0,0,32.0,0,A-C,32.0,1.0,0.0,0.0,1.0,1.0,NaN,1.0,NaN,0.0,1.0,0.0,0.0,NO CONTACTO,0,0,0
3269474,2021-06-22 22:31:35+00:00,2021-06-26 15:22:06+00:00,08055598,1570132,87949,319831,371.0,0.0,54,NaN,0,33126,0,32869,NaN,1,0,0,730.0,8,2,0,365.0,1,5,Comprar,32.0,Facebook Ads,0.0,0.0,32.0,0.0,32.0,0.0,0.0,1,32.0,0,A-C,32.0,1.0,1.0,32.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,NO CONTACTO,0,0,0
3269475,2021-08-16 12:55:54+00:00,2021-08-23 13:52:23+00:00,40641045,1666173,5760,608189,2512.0,81.0,40,607911.0,1361,30121,1361,31692,6079